In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from nnAudio.Spectrogram import CQT1992v2
from torch.utils.data import DataLoader, Dataset
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../")

In [ ]:
from src.transforms import stack_bandpass_transform, stack_bandpass_turkey_transform, stack_turkey, dumb_transform

In [ ]:
INPUT_PATH = Path("/home/trytolose/rinat/kaggle/grav_waves_detection/input")

In [ ]:
df = pd.read_csv(INPUT_PATH / "training_labels.csv")

files = list((INPUT_PATH / "train").rglob("*.npy"))
FILE_PATH_DICT = {x.stem: str(x) for x in files}
df["path"] = df["id"].apply(lambda x: FILE_PATH_DICT[x])

In [ ]:
class CFG:
    def __init__(self):
        self.qtransform_params = {
            'sr': 2048,
            'fmin': 30,
            'fmax': 400,
            'hop_length': 64,
            'bins_per_octave': 12,
            'filter_scale': 0.5,
        }

        self.bandpass_param = {
          'lf': 30,
          'hf': 400,
          'order': 8, 
          'sr': 2048,
        }
cfg = CFG()

In [ ]:
cqt = CQT1992v2(**cfg.qtransform_params)

In [ ]:

df = pd.read_csv("../top_min_max.csv")
df["path"] = df["id"].apply(lambda x: FILE_PATH_DICT[x])

In [ ]:
1/255

In [ ]:
x = np.load(df.loc[0, "path"])
# print(np.hstack(x)[:10])
# x = stack_bandpass_turkey_transform(x, cfg.bandpass_param)
# print(x[:10])
print(np.hstack(x)[-10:])
print(x.reshape(-1,)[-10:])

In [ ]:
for i in range(5):
    x = np.load(df.loc[i, "path"])
    print(x.std())
    x = stack_bandpass_turkey_transform(x, cfg.bandpass_param)
    print(x.std())
    x = cqt(torch.tensor(x).float())
    x = x.squeeze().numpy()
    plt.figure(figsize=(10,10))
    plt.imshow(x)
    plt.show()